In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

'''
Plot forecast maps with all available models.
'''

%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import numpy.ma as ma
import pandas as pd
import struct
import os
import xarray as xr
import glob
import datetime
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from esio import EsioData as ed
from esio import ice_plot
from esio import import_data
from esio import metrics
import dask
import xskillscore as xs

# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=.8, rc={"lines.linewidth": 2.5})

In [2]:
# from dask.distributed import Client
# client = Client(n_workers=2)
# client = Client()
# client
dask.config.set(scheduler='threads')  # overwrite default with threaded scheduler
# dask.config.set(scheduler='processes')  # overwrite default with threaded scheduler

In [3]:
# Parameters
pred_year = 2018 # Prediction year
Y_Start = 1979
Y_End = 2017

In [4]:
#############################################################
# Load in Data
#############################################################

E = ed.EsioData.load()
mod_dir = E.model_dir

# Get recent observations
ds_81 = xr.open_mfdataset(E.obs['NSIDC_0081']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True).sic
#ds_51 = xr.open_mfdataset(E.obs['NSIDC_0051']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True)
ds_79 = xr.open_mfdataset(E.obs['NSIDC_0079']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True).sic

In [5]:
# Fill in pole hole with sic of 1
ds_81 = ds_81.where(ds_81.hole_mask==0, other=1)
ds_79 = ds_79.where(ds_79.hole_mask==0, other=1)

In [6]:
ds_81.time[0].values, ds_81.time[-1].values

(numpy.datetime64('2015-01-01T00:00:00.000000000'),
 numpy.datetime64('2018-08-29T00:00:00.000000000'))

In [7]:
ds_79.time[0].values, ds_79.time[-1].values

(numpy.datetime64('1979-01-02T00:00:00.000000000'),
 numpy.datetime64('2017-12-31T00:00:00.000000000'))

In [8]:
# Combine bootstrap with NASA NRT
da_sic = ds_79.combine_first(ds_81)


In [9]:
# # Compute the rollowing 7 day window average (because we use the weekly forecast alhpa)
# Nday = 7
# ds_obs_avg = da_sic.rolling(min_periods=1, center=False, time=Nday).mean()
# ds_obs_avg.isel(time=100).plot()

In [10]:
# Reshape as year x doy
# Add DOY
DOY_all = [x.timetuple().tm_yday for x in pd.to_datetime(da_sic.time.values)]
da_sic.coords['doy'] = xr.DataArray(DOY_all, dims='time', coords={'time':da_sic.time})
year_all = [x.year for x in pd.to_datetime(da_sic.time.values)]
da_sic.coords['year'] = xr.DataArray(year_all, dims='time', coords={'time':da_sic.time})

In [11]:
da_sic.load()

<xarray.DataArray (time: 12876, y: 448, x: 304)>
array([[[ 0.,  0., ..., nan, nan],
        [ 0.,  0., ..., nan, nan],
        ...,
        [nan, nan, ...,  0.,  0.],
        [nan, nan, ...,  0.,  0.]],

       [[ 0.,  0., ..., nan, nan],
        [ 0.,  0., ..., nan, nan],
        ...,
        [nan, nan, ...,  0.,  0.],
        [nan, nan, ...,  0.,  0.]],

       ...,

       [[ 0.,  0., ..., nan, nan],
        [ 0.,  0., ..., nan, nan],
        ...,
        [nan, nan, ...,  0.,  0.],
        [nan, nan, ...,  0.,  0.]],

       [[ 0.,  0., ..., nan, nan],
        [ 0.,  0., ..., nan, nan],
        ...,
        [nan, nan, ...,  0.,  0.],
        [nan, nan, ...,  0.,  0.]]])
Coordinates:
  * time     (time) datetime64[ns] 1979-01-02 1979-01-04 1979-01-06 ...
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * y        (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
    lat      (x, y) float64 31.1 31.25 31.4 31.55 31.69 31.84 31.99 32.13

# Damped Anomaly 

In [12]:
cmod = 'dampedAnomalyTrend'
runType = 'forecast'

In [13]:
oceanmask = da_sic.isel(time=0).notnull().drop(['doy','year']).load()

In [14]:
# cdoy = 1
# # Select current day of year
# da_cdoy = da_sic.where(da_sic.doy==cdoy, drop=True).swap_dims({'time':'year'})
# # Select training years
# da_cdoy = da_cdoy.sel(year=slice(Y_Start, Y_End))    


In [15]:
# stopped on 56

# started on 204 to 211

### Calculate alpha, the lag1 auto correlation 

In [16]:
for cdoy in np.arange(1,da_sic.doy.max().values+1,1):
    print("Processing day of year", cdoy)
    
    #1) Get detrended SIC for curren doy and doy plus 7 days

    # Select current day of year
    da_cdoy = da_sic.where(da_sic.doy==cdoy, drop=True).swap_dims({'time':'year'})
    # Select training years
    da_cdoy = da_cdoy.sel(year=slice(Y_Start, Y_End))    
    # Remove trend
    print("    detrending current day...")
    da_detrend = metrics.detrend(da_cdoy.chunk({'year': -1}), 'year') # Have to rechunk year into one big one
    
#     plt.figure()
#     da_detrend.sel(year=2017).plot()
    
    # Calc the correlation between now and 7 days from now (simple option)
    day_lag = 7
    if cdoy+day_lag > da_sic.doy.max().values:
        next_doy = cdoy+day_lag - da_sic.doy.max().values
    else:
        next_doy = cdoy+day_lag
    da_cdoy_lag = da_sic.where(da_sic.doy==next_doy, drop=True).swap_dims({'time':'year'})
    da_cdoy_lag = da_cdoy_lag.sel(year=slice(Y_Start, Y_End))    
    
    # Remove trend
    print("    detrending lagged day...")
    da_detrend_lag = metrics.detrend(da_cdoy_lag.chunk({'year': -1}), 'year') # Have to rechunk year into one big one
    
#     plt.figure()
#     da_detrend_lag.sel(year=2017).plot()
    
    # Calculate correlation (over common years)
    common_years = np.intersect1d(da_detrend.year.values,da_detrend_lag.year.values)
    print("    Calculating pearson r...")
    alpha_cdoy = xs.pearson_r(da_detrend.sel(year=common_years), da_detrend_lag.sel(year=common_years), 'year')
    
    # nanpearson_r sets any pixel with only ocean in both arrays (SIC=0) to NaN becasue the std is zero.
    # Set those regions to alpha values of zero
    never_ice = ((alpha_cdoy.isnull()) & (oceanmask))
    alpha_cdoy = alpha_cdoy.where(~never_ice, other=0)
    
    alpha_cdoy.coords['lag'] = day_lag
    
#     plt.figure()
#     alpha_cdoy.plot()    
    
#     xr.exit()
    
#     # For each lag day
#     r_l = []
#     for day_lag in np.arange(1,3,1):
#         print(day_lag)
#         da_cdoy_lag = da_sic.where(da_sic.doy==cdoy+day_lag, drop=True).swap_dims({'time':'year'})
#         da_cdoy_lag = da_cdoy_lag.sel(year=slice(Y_Start, Y_End))    
#         # Remove trend
#         da_detrend_lag = metrics.detrend(da_cdoy_lag.chunk({'year': -1}), 'year') # Have to rechunk year into one big one
#         # Calculate correlation (over common years)
#         common_years = np.intersect1d(da_detrend.year.values,da_detrend_lag.year.values)
#         r = xs.pearson_r(da_detrend.sel(year=common_years), da_detrend_lag.sel(year=common_years), 'year')
#         r.coords['lag'] = day_lag
#         r_l.append(r)
#     alpha_cdoy = xr.concat(r_l, dim='lag')
    
    # Store
    alpha_cdoy.coords['doy'] = cdoy

    alpha_cdoy.name = 'alpha'
    
    alpha_cdoy = alpha_cdoy.drop(['time'])
        
    # Save to disk
    file_out = os.path.join(mod_dir, cmod, runType, 'param', str(pred_year)+'_'+format(cdoy, '03')+'_'+str(Y_Start)+'_'+str(Y_End)+'_Alpha.nc')
    alpha_cdoy.to_netcdf(file_out)
    print("Saved",file_out)
    

Processing day of year 1
    detrending current day...
    detrending lagged day...
    Calculating pearson r...


/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/dask/local.py:255: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


Saved /home/disk/sipn/nicway/data/model/dampedAnomalyTrend/forecast/param/2018_001_1979_2017_Alpha.nc
Processing day of year 2
    detrending current day...
    detrending lagged day...
    Calculating pearson r...


/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/dask/local.py:255: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


Saved /home/disk/sipn/nicway/data/model/dampedAnomalyTrend/forecast/param/2018_002_1979_2017_Alpha.nc
Processing day of year 3
    detrending current day...
    detrending lagged day...
    Calculating pearson r...


...

/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/dask/local.py:255: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


Saved /home/disk/sipn/nicway/data/model/dampedAnomalyTrend/forecast/param/2018_364_1979_2017_Alpha.nc
Processing day of year 365
    detrending current day...
    detrending lagged day...
    Calculating pearson r...


/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/dask/local.py:255: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


Saved /home/disk/sipn/nicway/data/model/dampedAnomalyTrend/forecast/param/2018_365_1979_2017_Alpha.nc
Processing day of year 366
    detrending current day...
    detrending lagged day...
    Calculating pearson r...
Saved /home/disk/sipn/nicway/data/model/dampedAnomalyTrend/forecast/param/2018_366_1979_2017_Alpha.nc


/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/dask/local.py:255: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


In [17]:
# from scipy import stats
# x = da_cdoy.sel(x=150,y=240).year.values
# y = da_cdoy.sel(x=150,y=240).values

# nas = np.logical_or(np.isnan(x), np.isnan(y))
# x = x[~nas]
# y = y[~nas]
# print(x,y)
# slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)


In [18]:
# from scipy import stats
# x = da_cdoy.sel(x=150,y=240).year
# y = da_cdoy.sel(x=150,y=240)

# nas = (x.notnull()) & (y.notnull())
# x = x.where(nas, drop=True)
# y = y.where(nas, drop=True)
# print(x.values,y.values)
# slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)


In [19]:
# alpha_cdoy = xr.open_mfdataset('/home/disk/sipn/nicway/data/model/dampedAnomaly/forecast/param/2018_*.nc',
#                               concat_dim='doy', autoclose=True)


# # Now make a forecast using todays SIC observations

# # Get most recent obs
# ds_81 = xr.open_mfdataset(E.obs['NSIDC_0081']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True)
# # Get mean sic by DOY
# mean_1980_2010_sic = xr.open_dataset(os.path.join(E.obs_dir, 'NSIDC_0051', 'agg_nc', 'mean_1980_2010_sic.nc')).sic
# mean_1980_2010_sic

# last_sic = ds_81.isel(time=-1).sic
# cdoy = metrics.get_DOY(last_sic.time)[0]
# cdoy

# # Calculate most recent anomaly
# c_anomaly = last_sic - mean_1980_2010_sic.sel(time=cdoy)

# plt.figure()
# last_sic.plot()



# da_l = []
# # Need to be careful that indices here are days (unless we change them)
# for fore_index in np.arange(1,30,5):
#     print(fore_index)
#     fore_anomaly = (alpha_cdoy.sel(doy=cdoy, lag=7).alpha**fore_index) * c_anomaly
#     fore_sic = fore_anomaly + mean_1980_2010_sic.sel(time=(cdoy+fore_index))
    
#     # Force prediction SIC to be between 0-1
#     ocnmask = fore_sic.notnull()
#     fore_sic = fore_sic.where(fore_sic >= 0, other=0).where(ocnmask)
#     fore_sic = fore_sic.where(fore_sic <= 1, other=1).where(ocnmask)
    
#     plt.figure()
#     fore_sic.plot()



# fore_anomaly
